In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import my_txtutils
import re
import nltk
mypath='./new_txts/'
txts='./txts/'

In [13]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(txts) if isfile(join(txts, f))]
print(onlyfiles)

['acts_new.txt', 'gal_eph_new.txt', 'heb_new.txt', 'jam_jud_new.txt', 'john_new.txt', 'jud_rev_new.txt', 'luke_8_john_new.txt', 'mark01_new.txt', 'matt02_new.txt', 'matt_new.txt', 'phil_col_new.txt', 'thes_tim_new.txt', 'tit_phl_new.txt']


In [13]:
#large files
def large_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'full.txt', 'w') as outfile:
        for fname in onlyfiles:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
#large files
def decimate_f(f):
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'tmp.txt', 'w') as outfile:
        with open(mypath+f) as infile:
            lc=0
            for line in infile:
                wrap=False
                cx=0
                tmp=""
                for i in line:
                    wrap=True if cx>99 else False
                    tmp+='\n' if (wrap==True and i==' ') else i
                    cx+=1
                outfile.write(tmp)
                lc+=1
    print(lc,'lines written')
decimate_f('full.txt')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [22]:
#small files
def small_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'full.txt', 'w') as outfile:
        for fname in onlyfiles:
            print('writing file',fname)
            with open(txts+fname) as infile:
                outfile.write(infile.read())
    return open(mypath+"full.txt").read()

In [24]:
txt=small_f()
words=re.sub(' +', ' ',txt.replace('\n',' ')).split(' ')
len(words)

writing file acts_new.txt
writing file gal_eph_new.txt
writing file heb_new.txt
writing file jam_jud_new.txt
writing file john_new.txt
writing file jud_rev_new.txt
writing file luke_8_john_new.txt
writing file mark01_new.txt
writing file matt02_new.txt
writing file matt_new.txt
writing file phil_col_new.txt
writing file thes_tim_new.txt
writing file tit_phl_new.txt


649190

In [25]:
len(txt)

6507438

In [15]:
# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

author=ok03

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(author+'.meta')
    new_saver.restore(sess, author)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    for i in range(1000):
        yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
        #print(tf.shape(yo))
        #print(len(yo))
        c = my_txtutils.sample_from_probabilities(yo, topn=2)
        y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(my_txtutils.convert_to_alphabet(c))
        print(c, end="")
        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            ncnt = 0


   o   d u k o   i n i   p i r i   s e   i   d u k o   o   p i r i a b e ,   o   b o   m i n e a   b
 u   d a p u   o r i   o r i a   o k w e i n m a   p i k i   n e m i k a s e   b e r e n i   n y a n
 a   a p u   m a   p i r i   o k u m a ,   o   n w o s e   i   b e m e ,   i   d u k o   o   p i r i
 a b e   i y a   a n i   i n e d a   o   b a r a   b i a   o k w e i n m a   n w o   d u k o   o   p
 i r i s a m ,   a n i a t i b i   o   b e r e n i   i   p o r i b i a   b a r a .   a n i   o k u m
 a   i n i   o   b a r a   s i k i m a   o k u m a   a n i a t i b i   a r i   i n e d a   i   s i  
 m i e m i e   y e   m i e   b u ,   o   b o   m a   n a   o   s e   m g b e s e c h i n   m i   o g
 o n o   b a k u b a b i a .   o k u m a   o m i n e a   b i e   s i m e   t a m u n o   b e   b e r
 e n i b i a   b o   g o y e g o y e   i   d a   b e   s e   o   p i r i b i a   b o r o s a m ,   o
 k u m a   o m i n e   i n e m a   m i e s e   o   t o m o n i k i r i   m i   b e   o k w 

In [35]:
def pcx(txt):
    # these must match what was saved !
    ALPHASIZE = my_txtutils.ALPHASIZE
    NLAYERS = 3
    INTERNALSIZE = 512

    ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
    ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

    author=ok03

    ncnt = 0
    with tf.Session() as sess:
        # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
        new_saver = tf.train.import_meta_graph(author+'.meta')
        new_saver.restore(sess, author)
        x = my_txtutils.convert_from_alphabet(ord(txt[0]))
        x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

        # initial values
        y = x
        h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
        arr=[]
        for i in range(len(txt)-1):
            yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
            c = my_txtutils.convert_from_alphabet(ord(txt[i+1]))
            d = my_txtutils.sample_from_probabilities(yo, topn=1)
            d = chr(my_txtutils.convert_to_alphabet(d))
            y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
            y1=np.reshape(yo,[ALPHASIZE,-1])
            y1=y1.flatten()
            #c = chr(my_txtutils.convert_to_alphabet(i))
            print(txt[i],':',c,':',y1[c],',max=',np.max(y1),"='%s' or %s"%(chr(my_txtutils.convert_to_alphabet(np.argmax(y1))),d))
            arr.append(y1[c])
    return np.log2(np.sum(arr))*(-1/len(txt))
    #return np.prod(arr))**(-1/len(txt)

In [36]:
t='''duko ini piri se i duko o piriabe, o bo minea b
u dapu ori oria okweinma piki nemikase bereni nyan
a apu ma piri okuma, o nwose i beme, i duko o piri
abe iya ani ineda o bara bia okweinma nwo duko o p
irisam, aniatibi o bereni i poribia bara. ani okum
a ini o bara sikima okuma aniatibi ari ineda i si 
miemie ye mie bu, o bo ma na o se mgbesechin mi og
ono bakubabia. okuma ominea bie sime tamuno be ber
enibia bo goyegoye i da be se o piribia borosam, o
kuma omine inema miese o tomonikiri mi be okwein'''
pcx(t)

d : 87 : 0.0 ,max= 0.0770766 =' ' or  
u : 77 : 0.0 ,max= 0.895978 =' ' or  
k : 81 : 0.0 ,max= 0.5329333 =' ' or  
o : 2 : 0.0 ,max= 0.99995935 =' ' or  
  : 75 : 0.0 ,max= 0.36144635 ='m' or m
i : 80 : 0.0 ,max= 0.9999676 =' ' or  
n : 75 : 0.0 ,max= 0.18348454 =' ' or  
i : 2 : 0.0 ,max= 0.9999759 =' ' or  
  : 82 : 0.0 ,max= 0.7913743 =' ' or  
p : 75 : 0.0 ,max= 0.9995127 =' ' or  
i : 84 : 0.0 ,max= 0.46761307 ='i' or i
r : 75 : 0.0 ,max= 0.9997508 =' ' or  
i : 2 : 0.0 ,max= 0.76876736 ='i' or i
  : 85 : 0.0 ,max= 0.9999703 =' ' or  
s : 71 : 0.0 ,max= 0.30323997 ='s' or s
e : 2 : 0.0 ,max= 0.9999521 =' ' or  
  : 75 : 0.0 ,max= 0.41298324 ='r' or r
i : 2 : 0.0 ,max= 0.9999945 =' ' or  
  : 70 : 0.0 ,max= 0.27091473 =' ' or  
d : 87 : 0.0 ,max= 0.9999479 =' ' or  
u : 77 : 0.0 ,max= 0.35302648 ='a' or a
k : 81 : 0.0 ,max= 0.9999987 =' ' or  
o : 2 : 0.0 ,max= 0.33377552 ='i' or i
  : 81 : 0.0 ,max= 0.99999106 =' ' or  
o : 2 : 0.0 ,max= 0.47501603 ='b' or b
  : 82 : 0.0 ,max= 0.

-0.006836573282877604

# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

author=ok03

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(author+'.meta')
    new_saver.restore(sess, author)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
    y=np.reshape(yo,[98,-1])
    y=y.flatten()
    v="hello world"
    for i in range(ALPHASIZE):
        #print(tf.shape(yo))
        #print(len(yo))
        #c = my_txtutils.sample_from_probabilities(yo, topn=2)
        #y = np.array([[c]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1
        c = chr(my_txtutils.convert_to_alphabet(i))
        print(c,':',y[i])
        if c == '\n':
            ncnt = 0
        else:
            ncnt += 1
        if ncnt == 100:
            print("")
            ncnt = 0

In [10]:
print('yes' if re.search("g", "abcdef") else 'no')

no


In [46]:
my_txtutils.convert_from_alphabet(ord('i'))

75

In [10]:
temp=''
'yes' if False else temp+='no'

SyntaxError: can't assign to conditional expression (<ipython-input-10-88533308bf58>, line 2)